!!! note

    This example is available as a jupyter notebook [here](https://github.com/simon-bachhuber/ring/blob/main/docs/notebooks/interactive_viewer.ipynb).

With the release of `imt-ring` >= `v1.7.0`, a CLI tool `ring-view` has been added.

>$ ring-view --help
```bash
NAME
    ring-view - View motion given by trajectory of minimal coordinates in interactive viewer.

SYNOPSIS
    ring-view PATH_SYS_XML <flags>

DESCRIPTION
    View motion given by trajectory of minimal coordinates in interactive viewer.

POSITIONAL ARGUMENTS
    PATH_SYS_XML
        Type: str
        Path to xml file defining the system.

FLAGS
    -p, --path_qs_np=PATH_QS_NP
        Type: Optional[Optional]
        Default: None
        Path to numpy array containing the timeseries of minimal coordinates with shape (T, DOF) where DOF is equal to `sys.q_size()`. Each minimal coordiante is from parent to child. So for example a `spherical` joint that connects the first body to the worldbody has a minimal coordinate of a quaternion that gives from worldbody to first body. The sampling rate of the motion is inferred from the `sys.dt` attribute. If `None` (default), then simply renders the unarticulated pose of the system.
    Additional flags are accepted.
```

Suppose, we have the following system stored in `pendulum.xml`
```xml
<x_xy model="double_pendulum">
    <options gravity="0 0 9.81" dt="0.01"/>
    <worldbody>
        <body name="upper" euler="0 90 0" joint="ry" damping="2" pos="0 0 2">
            <geom type="box" mass="10" pos="0.5 0 0" dim="1 0.25 0.2"/>
            <body name="lower" pos="1 0 0" joint="ry" damping="2">
                <geom type="box" mass="10" pos="0.5 0 0" dim="1 0.25 0.2"/>
            </body>
        </body>
    </worldbody>
</x_xy>
```

Then, we can directly execute and create an interactive view of the system using `ring-view pendulum.xml`
Alternatively, we can also work with the `InteractiveViewer` class.

In [19]:
from ring.extras.interactive_viewer import InteractiveViewer
import ring
import numpy as np

In [3]:
sys = ring.System.create("""
<x_xy model="double_pendulum">
    <options gravity="0 0 9.81" dt="0.01"/>
    <worldbody>
        <body name="upper" euler="0 90 0" joint="ry" damping="2" pos="0 0 2">
            <geom type="box" mass="10" pos="0.5 0 0" dim="1 0.25 0.2"/>
            <body name="lower" pos="1 0 0" joint="ry" damping="2">
                <geom type="box" mass="10" pos="0.5 0 0" dim="1 0.25 0.2"/>
            </body>
        </body>
    </worldbody>
</x_xy>
""")

In [4]:
viewer = InteractiveViewer(sys)

<img src="interactive_viewer1.png" alt="Viewer default pose" width="500">

Notice how this opens the interactive viewer window (where you can use the mouse to navigate, without blocking the main thread). So, we can update the view from this notebook.

In [8]:
viewer.update_q(np.array([-np.pi / 2, np.pi / 2]))

<img src="interactive_viewer2.png" alt="Viewer default pose" width="500">

This is really powerful for creating virtually any animation.

## Visualising the Motion that results from a `MotionConfig` object

In [20]:
sys = ring.io.load_example("test_three_seg_seg2")

In [28]:
_, (_, q, _, _) = ring.RCMG(sys, ring.MotionConfig(t_min=0.5, t_max=3, delta_ang_min=2, T=10.0, include_standstills_prob=0.2, pos_min_p3d_x=0.0, 
                                                   pos_min_p3d_y=0.0, pos_min_p3d_z=0.0), keep_output_extras=True).to_list(seed=2)[0]

executing generators: 100%|██████████| 1/1 [00:02<00:00,  2.95s/it]


In [29]:
# save the trajectory of minimal coordinates to disk
np.save("motion1", q)

Now, run `ring-view test_three_seg_seg2.xml motion1.np` to view the motion.

In [18]:
print(q.shape)

(2000, 9)
